# Generating counterfactuals for multi-class classification and regression models
This notebook will demonstrate how the DiCE library can be used for multiclass classification and regression for scikit-learn models. For demonstration, we will be using the genetic algorithm for CFs.

In [1]:
import dice_ml
from dice_ml import Dice

from dice_ml.utils import helpers # helper functions
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMClassifier, LGBMRegressor
import pandas as pd

We will use sklearn's internal datasets to demonstrate DiCE's features in this notebook

In [2]:
outcome_name = 'target'
# Function to process sklearn's internal datasets
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df[outcome_name] = pd.Series(sklearn_dataset.target)
    return df

## Multiclass Classification

For multiclass classification, we will use sklearn's Iris dataset. This data set consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset

In [3]:
from sklearn.datasets import load_iris
df_iris = sklearn_to_df(load_iris())
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [5]:
continuous_features_iris = df_iris.drop(outcome_name, axis=1).columns.tolist()
target = df_iris[outcome_name]

In [6]:
# Split data into train and test
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

datasetX = df_iris.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

categorical_features = x_train.columns.difference(continuous_features_iris) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_iris),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf_iris = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', LGBMClassifier())])
model_iris = clf_iris.fit(x_train, y_train)

In [7]:
d_iris = dice_ml.Data(dataframe=df_iris, 
                      continuous_features=continuous_features_iris, 
                      outcome_name=outcome_name)

# We provide the type of model as a parameter (model_type)
m_iris = dice_ml.Model(model=model_iris, backend="sklearn", model_type='classifier')

In [8]:
exp_genetic_iris = Dice(d_iris, m_iris, method="genetic")

As we can see below, all the target values will lie in the desired class

In [9]:
# Multiple queries can be given as input at once
query_instances_iris = x_train[2:3]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=4, desired_class = 2)
genetic_iris.visualize_as_dataframe()

Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 05 sec
Query instance (original outcome : 0)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.4,3.0,1.3,0.2,0



Diverse Counterfactual set (new outcome: 2)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.8,3.9,6.5,0.8,2
1,4.7,2.4,5.0,2.4,2
2,4.4,2.9,2.6,2.3,2
3,4.6,3.7,4.6,2.0,2


In [10]:
# Multiple queries can be given as input at once
query_instances_iris = x_train[17:19]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=7, desired_class = 2)
genetic_iris.visualize_as_dataframe(show_only_changes=True)

Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 08 sec
Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 09 sec
Query instance (original outcome : 1)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.7,2.9,4.2,1.3,1



Diverse Counterfactual set (new outcome: 2)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.6,2.8,5.4,1.0,2.0
1,5.0,3.0,6.9,0.7,2.0
2,5.0,2.4,6.1,0.9,2.0
3,6.9,3.1,6.1,-,2.0
4,5.5,2.1,6.0,1.8,2.0
5,5.8,3.1,3.9,2.0,2.0
6,7.1,3.0,6.4,1.7,2.0


Query instance (original outcome : 1)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,6.3,3.3,4.7,1.6,1



Diverse Counterfactual set (new outcome: 2)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,6.2,3.7,6.0,1.4,2.0
1,4.7,3.4,6.5,2.4,2.0
2,6.0,3.7,6.5,1.9,2.0
3,6.1,3.9,5.5,1.5,2.0
4,-,3.5,5.1,1.7,2.0
5,5.4,3.0,2.3,2.2,2.0
6,6.9,3.6,6.5,1.2,2.0


# Regression

For regression, we will use sklearn's boston dataset. This dataset contains boston house-prices. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset

In [11]:
from sklearn.datasets import load_boston
df_boston = sklearn_to_df(load_boston())
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [12]:
df_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [13]:
continuous_features_boston = df_boston.drop(outcome_name, axis=1).columns.tolist()
target = df_boston[outcome_name]

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

# Split data into train and test
datasetX = df_boston.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0)

categorical_features = x_train.columns.difference(continuous_features_boston) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_boston),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
regr_boston = Pipeline(steps=[('preprocessor', transformations),
                      ('regressor', LGBMRegressor())])

In [15]:
d_boston = dice_ml.Data(dataframe=df_boston, continuous_features=continuous_features_boston, outcome_name=outcome_name)
model_boston = regr_boston.fit(x_train, y_train)
# We provide the type of model as a parameter (model_type)
m_boston = dice_ml.Model(model=model_boston, backend="sklearn", model_type='regressor')

In [16]:
exp_genetic_boston = Dice(d_boston, m_boston, method="genetic")

As we can see below, all the target values will lie in the desired range

In [17]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[2:3]
genetic_boston = exp_genetic_boston.generate_counterfactuals(query_instances_boston, 
                                                             total_CFs=2, 
                                                             desired_range=[30, 45])
genetic_boston.visualize_as_dataframe(show_only_changes=True)


Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 02 sec
Query instance (original outcome : 24)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.11329,30.0,4.93,0.0,0.428,6.897,54.3,6.3361,6.0,300.0,16.6,391.25,11.38,23.909689



Diverse Counterfactual set (new outcome: [30, 45])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,7.64683,52.1,9.9,-0.1,0.523,8.705,36.3,6.6281,20.7,436.8,20.1,276.4,8.26,31.655
1,5.02885,77.0,6.2,-0.1,0.751,5.091,94.9,7.2368,8.8,265.3,15.4,387.4,3.29,30.972


In [18]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[17:19]
genetic_boston = exp_genetic_boston.generate_counterfactuals(query_instances_boston, total_CFs=4, desired_range=[40, 50])
genetic_boston.visualize_as_dataframe(show_only_changes=True)

Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...


Diverse Counterfactuals found! total time taken: 00 min 06 sec
Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 06 sec
Query instance (original outcome : 46)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.01501,90.0,1.21,1.0,0.401,7.923,24.8,5.885,1.0,198.0,13.6,395.52,3.16,46.350821



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,10.60305,23.3,2.9,0.2,0.82,7.82,80.2,6.1631,7.7,256.3,14.6,278.9,5.67,43.825
1,8.48339,72.6,27.7,0.5,0.642,8.036,57.6,4.7803,8.4,690.6,14.5,162.4,4.28,49.793
2,4.73671,53.3,15.8,0.1,0.647,8.417,48.5,10.5646,21.4,411.5,18.3,383.5,-,42.963
3,16.33002,67.1,3.5,0.2,0.852,7.473,-,4.2948,16.5,610.6,16.7,249.7,-,41.251


Query instance (original outcome : 31)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.06911,45.0,3.44,0.0,0.437,6.739,30.8,6.4798,5.0,398.0,15.2,389.71,4.69,31.00116



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,10.08553,7.0,1.8,-,0.538,8.114,9.5,3.5723,21.9,242.9,13.7,343.4,5.97,46.256
1,45.71121,34.1,7.3,-,0.749,8.021,79.2,3.6388,23.1,209.8,17.5,158.2,4.64,41.481
2,2.46364,6.6,19.3,-0.1,0.821,7.947,30.7,5.8394,2.7,196.0,14.5,58.7,7.58,41.523
3,0.65003,30.9,23.8,0.8,0.405,8.509,13.6,2.6086,15.7,668.9,21.3,283.4,4.52,41.123
